<a href="https://colab.research.google.com/github/Ramykaz/Solar-Panel-Anomaly-Detection/blob/main/PV_IR_Anomaly_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP-BY-STEP CELL-BASED DEFECT CLASSIFICATION PIPELINE
# ------------------------------------------------------
# This script does the following:
# 1. Loads a full panel image (grayscale or color)
# 2. Uses a YOLOv8 model to detect cells in the panel
# 3. Crops each detected cell and resizes to 224x224
# 4. Sends each crop to a trained YOLOv8 classifier model
# 5. Displays and saves final image with bounding boxes and defect labels

import os
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Define paths to the models
cell_detector_path = "/content/roboflow_cell_detector.pt"  # replace with actual path
classifier_path = "/content/runs/classify/fault_11class_model/weights/best.pt"  # replace with actual path
input_image_path = "/content/panel_image.jpg"  # your panel image to test

# Create temp output folder for cell crops
os.makedirs("cell_crops", exist_ok=True)

# STEP 1: Load the full panel image
original = cv2.imread(input_image_path)
if original is None:
    raise ValueError("Image not found or invalid path!")

# Convert to grayscale if not already
gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)

# Save grayscale for visualization/debugging
cv2.imwrite("gray_input.jpg", gray)

# STEP 2: Run the Cell Detector YOLOv8 model
print("🔍 Detecting cells...")
cell_detector = YOLO(cell_detector_path)
detection_results = cell_detector("gray_input.jpg")[0]

# STEP 3: Crop each detected cell and resize to 224x224 for classification
print("✂️  Cropping and resizing cells...")
cell_crops = []
coords = []
for i, box in enumerate(detection_results.boxes.xyxy):
    x1, y1, x2, y2 = map(int, box)
    crop = gray[y1:y2, x1:x2]
    resized_crop = cv2.resize(crop, (224, 224))
    crop_path = f"cell_crops/cell_{i}.jpg"
    cv2.imwrite(crop_path, resized_crop)
    cell_crops.append(crop_path)
    coords.append((x1, y1, x2, y2))

# STEP 4: Run the classifier on each cell crop
print("🧠 Running classification on cells...")
classifier = YOLO(classifier_path)
predictions = []
for crop_path in cell_crops:
    pred_result = classifier.predict(crop_path, imgsz=224, verbose=False)[0]
    class_name = pred_result.names[pred_result.probs.top1]
    confidence = pred_result.probs.top1conf
    predictions.append((class_name, confidence))

# STEP 5: Annotate the original image with defect labels
print("🖍️ Annotating results on the image...")
annotated = original.copy()
for (x1, y1, x2, y2), (label, conf) in zip(coords, predictions):
    color = (0, 255, 0) if "No" in label or "normal" in label.lower() else (0, 0, 255)
    cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
    cv2.putText(annotated, f"{label} ({conf:.2f})", (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)

# Save and display the result
output_path = "annotated_output.jpg"
cv2.imwrite(output_path, annotated)

# Show image inline
plt.figure(figsize=(12, 12))
plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Detected and Classified Cells")
plt.show()

print("✅ Pipeline completed. Output saved as:", output_path)
